In [1]:
import autograd.numpy as np
from autograd import grad

In [10]:
class NANDNeuron():
    def __init__(self, n):
        self.n = n
        self.epsilons = np.full((n), 0.0) #np.random.uniform(-1.0, 1.001, n)#
        self.effector = -1
        
    def __repr__(self):
        return str(self.epsilons)
        
    def __beforePresent__(self):
        self.effector = -1
        self.delta = 0
    
    def getDelta(self):
        return self.delta
    
    def setDelta(self, delta):
        self.delta = delta
    
    def present(self, inputs):
        self.__beforePresent__()
        
        self.inputs = inputs
        
        mus = np.array(list(map(lambda x: self.epsilons[x] + inputs[x] - self.epsilons[x] * inputs[x], range(0, len(inputs)))))
        self.mus = mus
#         mus = np.array(list(map(lambda x: np.power(inputs[x], (1-self.epsilons[x])), range(0, len(inputs)))))
        z = np.product(mus)
        
        y = 1 - z#np.power(np.e, z)
        self.output = y
        
        return self.output

    def getInput(self):
        return self.inputs
    
    def getMus(self):
        return self.mus
    
    def getOutput(self):
        return self.output
    
    def updateWeights(self, grad):
        self.epsilons = self.epsilons - grad
#         for i in range(0, len(self.epsilons)):
#             if self.epsilons[i] > 1:
#                 self.epsilons[i] = 1
#             elif self.epsilons[i] < 0:
#                 self.epsilons[i] = 0
    
    def getWeights(self):
        return self.epsilons
    
    def getEffector(self):
        return self.effector

class NANDLayer():
    def __init__(self, inputs, nodes):
        self.layer = []
        
        for i in range(0, nodes):
            self.layer.append(NANDNeuron(inputs))
            
    def __repr__(self):
        s = ""
        for l in self.layer:
            s += (str(l) + " ,")
            
        return s

    def getLayer(self):
        return self.layer
    
    def setFolowingLayer(self, l):
        self.folowingLayer = l
        
    def present(self, inputs):
        out = []
        for n in self.layer:
            out.append(n.present(inputs))
        
        return out
    
    def backprop(self, prediction, target, output=False):
        for n in range(0, len(self.layer)):
            grad = None
            if output:
                grad = gradientOutputLayer(self.layer[n], target, prediction)
            else:
                grad = gradientHiddenLayer(self.layer[n], n, self.folowingLayer)
                
            self.layer[n].updateWeights(grad * 0.03)
    
class NANDNetwork():
    def __init__(self, nIns, lParams, nOuts):
        self.layers = []
        
        lParams.append(nOuts)
        inputs = nIns
        for l in lParams:
            self.layers.append(NANDLayer(inputs, l))
            inputs = l
                    
        for i in range(1, len(self.layers)):
            self.layers[i-1].setFolowingLayer(self.layers[i])
        
    def __repr__(self):
        s = ""
        for l in range(0, len(self.layers)):
            s += ("Layer " + str(l+1) + " -> " + str(self.layers[l]) + "\n")
            
        return s
        
    def fowardprop(self, inputs):
        for l in self.layers:
            inputs = l.present(inputs)
            
        return inputs[0]
            
    def backprop(self, prediction, target):
        for i in range(len(self.layers)-1, -1, -1):
            layer = self.layers[i]
            layer.backprop(prediction, target, i==(len(self.layers) - 1))
    
    
def MSE(network, data, targets, p=False):
    expected = np.array(list(map(lambda x: network.fowardprop(x), data)))
    if p:
        print(expected)
    return (1.0/2.0) * np.sum(np.power(np.subtract(expected, targets), 2))

def gradientHiddenLayer(neuron, neuronNumber, folowingLayer):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)

    delta = 0
    for i in range(0, len(folowingLayer.getLayer())):
        fn = folowingLayer.getLayer()[i]
        delta += fn.getDelta() * -(1-fn.getWeights()[neuronNumber]) * np.product(np.delete(fn.getMus(), neuronNumber))
        
    neuron.setDelta(delta)
    
    for i in range(0, numWeights):
        grad[i] = delta * -(1-neuron.getInput()[i]) * np.product(np.delete(neuron.getMus(), i))
    
    return grad
    
    
def gradientOutputLayer(neuron, target, prediction):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)

    delta = -(target - prediction)
    for i in range(0, numWeights):
        grad[i] = delta * -(1-neuron.getInput()[i]) * np.product(np.delete(neuron.getMus(), i))
        
    neuron.setDelta(delta)
    
    return grad


In [11]:
def trainNANDNetwork(data, targets, inputNodes, hLayers, outNodes):
    network = NANDNetwork(inputNodes, hLayers, outNodes)
    print(network)
    print("Initial Loss: ", MSE(network, data, targets))
    
    for i in range(1, 1000):
#         if (MSE(network, data, targets) < 0.0000000000001):
#             break
#         if i%1000 == 0:
#             print("Iteration -> " + str(i))
            
        for j in range(0, len(data)):
            prediction = network.fowardprop(data[j])
            network.backprop(prediction, targets[j])
            
    print("Trained Loss: ", MSE(network, data, targets, True))
    return network
            
            

In [12]:
# NAND Gate Data
# data = np.array([[0.0,0.0,0.0],[1.0,0.0,0.0],[0.0,1.0,0.0],[1.0,1.0,0.0]])
# targets = np.array([1.0,1.0,1.0,0.0])



# NOT Gate Data
# data = np.array([[0.0, 1.0, 0.0], [1.0, 1.0, 0.0]])
# targets = np.array([1.0, 0.0])
# trainNANDNetwork(data, targets, 3, [], 1)


# AND Gate Data
# data = np.array([[0.0, 0.0, 0.0], [0.0, 1.0,0.0], [1.0, 0.0,0.0], [1.0, 1.0,0.0]])
# targets = np.array([0.0, 0.0, 0.0, 1.0])

# OR Gate Data 
data = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targets = np.array([0.0, 1.0, 1.0, 1.0])

trainNANDNetwork(data, targets, 2, [3], 1)


Layer 1 -> [ 0.  0.] ,[ 0.  0.] ,[ 0.  0.] ,
Layer 2 -> [ 0.  0.  0.] ,

Initial Loss:  1.0
[ 0.31702291  0.80133466  0.80116403  1.09287463]
Trained Loss:  0.0940664415145


Layer 1 -> [ 0.28667572  0.28656078] ,[ 0.28667572  0.28656078] ,[ 0.28667572  0.28656078] ,
Layer 2 -> [-0.4528618 -0.4528618 -0.4528618] ,